In [1]:
!pip install -r requirements.txt

  Using cached text_hammer-0.1.5-py3-none-any.whl (7.6 kB)
  Obtaining dependency information for spacy from https://files.pythonhosted.org/packages/d6/9e/8afc618cfed4b5dc602b11754d4d9193a268439704defae301bffca7f04c/spacy-3.6.1-cp311-cp311-win_amd64.whl.metadata
  Using cached spacy-3.6.1-cp311-cp311-win_amd64.whl.metadata (26 kB)
  Obtaining dependency information for tensorflow from https://files.pythonhosted.org/packages/9e/b8/ed5f794359d05cd0bffb894c6418da87b93016ee17b669d55c45d1bd5d5b/tensorflow-2.13.0-cp311-cp311-win_amd64.whl.metadata
  Using cached tensorflow-2.13.0-cp311-cp311-win_amd64.whl.metadata (2.6 kB)
  Obtaining dependency information for datasets from https://files.pythonhosted.org/packages/09/7e/fd4d6441a541dba61d0acb3c1fd5df53214c2e9033854e837a99dd9e0793/datasets-2.14.5-py3-none-any.whl.metadata
  Using cached datasets-2.14.5-py3-none-any.whl.metadata (19 kB)
  Obtaining dependency information for tensorflow-intel==2.13.0 from https://files.pythonhosted.org/packages

In [2]:
import opendatasets as od

od.download("https://www.kaggle.com/datasets/kazanova/sentiment140")

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username:Your Kaggle Key:Downloading sentiment140.zip to .\sentiment140


100%|██████████| 80.9M/80.9M [00:04<00:00, 18.2MB/s]


In [1]:
import pandas as pd
df = pd.read_csv(r"C:\Users\kausikdevanathan\DataspellProjects\sentimentAnalysis\sentiment140\training.1600000.processed.noemoticon.csv",names=["sentiment","id","date","query","user","text"],encoding='latin-1')
df.head()

,sentiment,id,date,query,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [2]:
df.isnull().sum()

sentiment    0
id           0
date         0
query        0
user         0
text         0
dtype: int64

In [3]:
df.drop(["id","date","query","user"],axis=1,inplace=True)

In [4]:
df.head()

,sentiment,text
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,is upset that he can't update his Facebook by ...
2,0,@Kenichan I dived many times for the ball. Man...
3,0,my whole body feels itchy and like its on fire
4,0,"@nationwideclass no, it's not behaving at all...."


In [2]:
import transformers
from transformers import AutoTokenizer,pipeline,AutoModelForSequenceClassification
model_path = "cardiffnlp/twitter-roberta-base-sentiment"
sentiment = pipeline("sentiment-analysis", model=model_path, tokenizer=model_path)

RuntimeError: Failed to import transformers.pipelines because of the following error (look up to see its traceback):
cannot import name 'ARRAY_FUNCTIONS' from 'numpy.core.overrides' (C:\Users\kausikdevanathan\anaconda3\envs\SIH\Lib\site-packages\numpy\core\overrides.py)

In [6]:
sentiment("I am very happy")

[{'label': 'LABEL_2', 'score': 0.9876530170440674}]

In [7]:
sentiment("I am very sad")

[{'label': 'LABEL_0', 'score': 0.949698805809021}]

In [8]:
sentiment("I am very angry")

[{'label': 'LABEL_0', 'score': 0.9702324867248535}]

In [9]:
# !python -m spacy download en_core_web_sm -q
# Run this if you don't have spacy installed

In [10]:
# %%time
# import text_hammer as th
# from tqdm._tqdm_notebook import tqdm_notebook
# tqdm_notebook.pandas()
# def text_preprocessing(df,col_name):
#     column = col_name
#     df[column] = df[column].progress_apply(lambda x:str(x).lower())
#     #     df[column] = df[column].progress_apply(lambda x: th.cont_exp(x))
#     #you're -> you are; i'm -> i am
#     df[column] = df[column].progress_apply(lambda x: th.remove_emails(x))
#     df[column] = df[column].progress_apply(lambda x: th.remove_html_tags(x))
#     df[column] = df[column].progress_apply(lambda x: th.remove_special_chars(x))
#     df[column] = df[column].progress_apply(lambda x: th.remove_accented_chars(x))
#     df[column] = df[column].progress_apply(lambda x: th.make_base(x)) #ran -> run,
#     return(df)
# 
# df['text'] = text_preprocessing(df,'text')

In [11]:
# import multiprocessing
# import time
# 
# def text_preprocessing_worker(text):
#     """Preprocess text using a specific function."""
# 
#     # Preprocess the text using your chosen function
#     preprocessed_text = th.cont_exp(text)
# 
#     return preprocessed_text
# 
# # Create a pool of workers to execute the text preprocessing function in parallel
# start = time.time()
# pool = multiprocessing.Pool(4)
# 
# # Apply the text preprocessing function to each text in the DataFrame using the pool of workers
# preprocessed_texts = pool.map(text_preprocessing_worker, df['text'])
# 
# # Add the preprocessed texts to the DataFrame
# df['text'] = preprocessed_texts
# 
# # Close the pool of workers
# pool.close()
# print(time.time() - start)


In [3]:
%%time
import unicodedata
from tqdm._tqdm_notebook import tqdm_notebook

def vectorized_text_preprocessing_tqdm(df, col_name):
    """
    Performs text preprocessing on a Pandas DataFrame column using vectorized operations and shows the progress bar.

    Args:
        df: A Pandas DataFrame.
        col_name: The name of the column to preprocess.

    Returns:
        A Pandas DataFrame with the preprocessed column.
    """

    # Show the progress bar.
    tqdm_notebook.pandas()

    # Convert the column to lowercase.
    df[col_name] = df[col_name].progress_apply(lambda x: x.lower())

    # Remove emails, HTML tags, special characters, and accented characters.
    df[col_name] = df[col_name].progress_apply(lambda x: x.replace(r"[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+", ""))
    df[col_name] = df[col_name].progress_apply(lambda x: x.replace(r"<[^>]*>", ""))
    df[col_name] = df[col_name].progress_apply(lambda x: x.replace(r"[^\w\s]", ""))
    df[col_name] = df[col_name].progress_apply(lambda x: unicodedata.normalize("NFKD", x).encode("ascii", errors="ignore").decode("ascii"))

    # Make the text base form.
    df[col_name] = df[col_name].progress_apply(lambda x: " ".join(x.split()))

    return df



# Preprocess the text column using the vectorized approach with progress bar.
df['text'] = vectorized_text_preprocessing_tqdm(df, "text")

<timed exec>:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.*` instead of `tqdm._tqdm_notebook.*`


  0%|          | 0/1600000 [00:00<?, ?it/s]

  0%|          | 0/1600000 [00:00<?, ?it/s]

  0%|          | 0/1600000 [00:00<?, ?it/s]

  0%|          | 0/1600000 [00:00<?, ?it/s]

  0%|          | 0/1600000 [00:00<?, ?it/s]

  0%|          | 0/1600000 [00:00<?, ?it/s]

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "C:\Users\kausikdevanathan\anaconda3\envs\SIH\Lib\site-packages\IPython\core\magics\execution.py", line 1340, in time
    exec(code, glob, local_ns)
  File "<timed exec>", line 36, in <module>
  File "C:\Users\kausikdevanathan\anaconda3\envs\SIH\Lib\site-packages\pandas\core\frame.py", line 4084, in __setitem__
    self._set_item_frame_value(key, value)
  File "C:\Users\kausikdevanathan\anaconda3\envs\SIH\Lib\site-packages\pandas\core\frame.py", line 4212, in _set_item_frame_value
    raise ValueError("Columns must be same length as key")
ValueError: Columns must be same length as key

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\kausikdevanathan\anaconda3\envs\SIH\Lib\site-packages\IPython\core\interactiveshell.py", line 2120, in showtraceback
    stb = self.InteractiveTB.structured_traceback(
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\kausikde

In [13]:
df.to_csv(r"C:\Users\kausikdevanathan\DataspellProjects\sentimentAnalysis\sentiment140\preprocessed.csv")

In [14]:
df['sentiment'].value_counts()

sentiment
0    800000
4    800000
Name: count, dtype: int64

In [4]:
from sklearn.model_selection import train_test_split,KFold,StratifiedKFold
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix

X = df["text"]
y = df["sentiment"]

X_train, X_test_val, y_train, y_test_val = train_test_split(X, y, test_size=0.30, random_state=42)

# Split the test-validation set into test and validation sets
X_test, X_val, y_test, y_val = train_test_split(X_test_val, y_test_val, test_size=0.50, random_state=42)

# Save the train, test, and validation sets to disk
X_train.to_csv(r"C:\Users\kausikdevanathan\DataspellProjects\sentimentAnalysis\train_val_test_data\train.csv", index=False)
X_test.to_csv(r"C:\Users\kausikdevanathan\DataspellProjects\sentimentAnalysis\train_val_test_data\test.csv", index=False)
X_val.to_csv(r"C:\Users\kausikdevanathan\DataspellProjects\sentimentAnalysis\train_val_test_data\validation.csv", index=False)
y_train.to_csv(r"C:\Users\kausikdevanathan\DataspellProjects\sentimentAnalysis\train_val_test_data\train_labels.csv", index=False)
y_test.to_csv(r"C:\Users\kausikdevanathan\DataspellProjects\sentimentAnalysis\train_val_test_data\test_labels.csv", index=False)
y_val.to_csv(r"C:\Users\kausikdevanathan\DataspellProjects\sentimentAnalysis\train_val_test_data\validation_labels.csv", index=False)

ImportError: cannot import name 'ARRAY_FUNCTIONS' from 'numpy.core.overrides' (C:\Users\kausikdevanathan\anaconda3\envs\SIH\Lib\site-packages\numpy\core\overrides.py)

In [20]:
import datasets

dataset = datasets.load_dataset(
    "csv",
    data_files={
        "train": r"C:\Users\kausikdevanathan\DataspellProjects\sentimentAnalysis\train_val_test_data\train.csv",
        "test": r"C:\Users\kausikdevanathan\DataspellProjects\sentimentAnalysis\train_val_test_data\test.csv",
        "validation": r"C:\Users\kausikdevanathan\DataspellProjects\sentimentAnalysis\train_val_test_data\validation.csv",
    },
    column_names=["text", "label"],
    cache_dir="",
)



In [17]:
dataset['train']

Dataset({
    features: ['text', 'label'],
    num_rows: 1120001
})

In [35]:
from transformers import Trainer,enable_full_determinism
enable_full_determinism(seed=42)

# dataset.full_determinism = False
trainer = Trainer(
    model=model_path,
    args={
        "num_train_epochs": 3,
        "learning_rate": 5e-5,
        "evaluation_strategy": "epoch",
        "per_device_train_batch_size": 32,
        "per_device_eval_batch_size": 32,
        "logging_steps": 100,
        "weight_decay": 0.01,
        "load_best_model_at_end": True,
        "metric_for_best_model": "accuracy",
    },
    train_dataset=dataset["train"],
    eval_dataset=dataset["validation"],
    tokenizer=model_path
)

AttributeError: 'dict' object has no attribute 'full_determinism'

In [37]:
import transformers

from transformers import AutoTokenizer, TrainingArguments, Trainer,enable_full_determinism

enable_full_determinism(seed=42)

model_path = "cardiffnlp/twitter-roberta-base-sentiment"

# Load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSequenceClassification.from_pretrained(model_path)

# Set the training arguments
training_args = TrainingArguments(
    output_dir="models",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    num_train_epochs=3,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    logging_steps=100,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
)

# Create a trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["validation"],
)

# Train the model
trainer.train()

# Evaluate the model
trainer.evaluate()


ValueError: The batch received was empty, your model won't be able to train on it. Double-check that your training dataset contains keys expected by the model: input_ids,attention_mask,token_type_ids,position_ids,head_mask,inputs_embeds,labels,output_attentions,output_hidden_states,return_dict,label,labels,label_ids.

In [38]:
from transformers import TrainingArguments, Trainer, enable_full_determinism
enable_full_determinism(seed=42)

# Set the training arguments
training_args = TrainingArguments(
    output_dir="models",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    num_train_epochs=3,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    logging_steps=100,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
)

# Create a trainer
trainer = Trainer(
    model=sentiment,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["validation"],
)

# Train the model
trainer.train()

# Evaluate the model
trainer.evaluate()


AttributeError: 'TextClassificationPipeline' object has no attribute 'to'

In [ ]:
!pip install accelerate -U 

In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("cardiffnlp/twitter-roberta-base-sentiment-latest")
model = AutoModelForSequenceClassification.from_pretrained("cardiffnlp/twitter-roberta-base-sentiment-latest")

In [ ]:
model_path = "cardiffnlp/twitter-roberta-base-sentiment-latest"

In [ ]:
# import datasets
# dataset = datasets.load_dataset("sentiment140", split="train")
# import datasets

# dataset = datasets.load_dataset("sentiment140",encode="latin-1", split="train")
dataset = datasets.load_dataset(
    "csv",
    data_files={
        "train": r"C:\Users\kausikdevanathan\DataspellProjects\sentimentAnalysis\train_val_test_data\train.csv",
        "test": r"C:\Users\kausikdevanathan\DataspellProjects\sentimentAnalysis\train_val_test_data\test.csv",
        "validation": r"C:\Users\kausikdevanathan\DataspellProjects\sentimentAnalysis\train_val_test_data\validation.csv",
    },
    column_names=["text", "label"],
    cache_dir="",
)
# tokenizer = transformers.AutoTokenizer.from_pretrained("cardiffnlp/twitter-roberta-base-sentiment",padding=True, truncation=True, max_length=128)

preprocessed_dataset = []
for tweet, label in zip(dataset["train"]["text"], dataset["validation"]["label"]):
    input_ids = tokenizer(tweet, return_tensors="pt").input_ids
    preprocessed_dataset.append({"input_ids": input_ids, "labels": label})


In [1]:
!pip install GPUtil -q
import GPUtil

for gpu in GPUtil.getGPUs():
    print("GPU Name:", gpu.name)
    print("GPU Memory Total [GB]:", gpu.memoryTotal / 1e9)


GPU Name: Tesla T4
GPU Memory Total [GB]: 1.536e-05


In [2]:
!pip install torch torchvision torchaudio cudatoolkit

  Using cached torchvision-0.15.2-cp311-cp311-win_amd64.whl (1.2 MB)
  Using cached torchaudio-2.0.2-cp311-cp311-win_amd64.whl (2.1 MB)


ERROR: Could not find a version that satisfies the requirement cudatoolkit (from versions: none)
ERROR: No matching distribution found for cudatoolkit


In [39]:
import torch

print("GPU Device Name:", torch.cuda.get_device_name(0))
print("GPU Device Total Memory [GB]:", torch.cuda.get_device_properties(0).total_memory / 1e9)


AssertionError: Torch not compiled with CUDA enabled

In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="models",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    num_train_epochs=3,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    logging_steps=100,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy"
)

trainer = Trainer(
model=model,
args=training_args,
train_dataset=preprocessed_dataset,
)

trainer.train()


In [ ]:
# Setting up the device for GPU usage

from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

In [ ]:
device